<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [1]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [4]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [35]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

OperationalError: table houses already exists

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [36]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute('INSERT into houses values (?,?,?,?,?)',last_sale )
# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [37]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
c.executemany ('INSERT into houses values (?,?,?,?,?)',recent_sales)

conn.commit()

Select all rows from houses

In [111]:
m=c.execute('SELECT * FROM houses')
m.fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000),
 (5, 4000, 5, 22, 619000),
 (6, 2390, 4, 34, 319000),
 (7, 1870, 3, 14, 289000),
 (8, 1505, 3, 90, 269000)]

In [39]:
for row in c.execute ('SELECT * FROM houses'):
    print(row)

(1, 4000, 5, 22, 619000)
(2, 2390, 4, 34, 319000)
(3, 1870, 3, 14, 289000)
(4, 1505, 3, 90, 269000)
(5, 4000, 5, 22, 619000)
(6, 2390, 4, 34, 319000)
(7, 1870, 3, 14, 289000)
(8, 1505, 3, 90, 269000)





#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [44]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('houses.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [45]:
data

[[3000, 4, 30, 50000], [2300, 3, 30, 45000]]

In [46]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [47]:
# check:
print(type(data))
print(data[0:5])

<class 'list'>
[[None, 3000, 4, 30, 50000], [None, 2300, 3, 30, 45000]]


Now we can insert each list item as a row of fields in the database.

In [48]:
# loop through data, running an INSERT on each record (i.e. sublist):
d = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In [51]:
for i in c.execute('select * from houses'):
    print(i)

(1, 4000, 5, 22, 619000)
(2, 2390, 4, 34, 319000)
(3, 1870, 3, 14, 289000)
(4, 1505, 3, 90, 269000)
(5, 4000, 5, 22, 619000)
(6, 2390, 4, 34, 319000)
(7, 1870, 3, 14, 289000)
(8, 1505, 3, 90, 269000)
(9, 3000, 4, 30, 50000)
(10, 2300, 3, 30, 45000)


In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [0]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [52]:
c.execute('DELETE FROM houses WHERE field1 IN (9, 10)')
conn.commit()

In [53]:
for i in c.execute('select * from houses'):
    print(i)

(1, 4000, 5, 22, 619000)
(2, 2390, 4, 34, 319000)
(3, 1870, 3, 14, 289000)
(4, 1505, 3, 90, 269000)
(5, 4000, 5, 22, 619000)
(6, 2390, 4, 34, 319000)
(7, 1870, 3, 14, 289000)
(8, 1505, 3, 90, 269000)


QUIZ: What would `DELETE FROM houses` do?

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [55]:
rm4=c.execute('select * from houses where bdrms = 4')
rm4.fetchall()

[(2, 2390, 4, 34, 319000), (6, 2390, 4, 34, 319000)]

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [56]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(3, 1687.5, 279000.0), (4, 2390.0, 319000.0), (5, 4000.0, 619000.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [57]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [59]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('houses.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,3000,4,30,50000
1,2300,3,30,45000
2,4000,4,25,65000
3,2000,3,20,45000
4,4500,5,26,75000


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [60]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [69]:
AVG = c.execute('select avg(price) as avg_pr, bdrms from houses_pandas group by bdrms')
AVG.fetchall()

[(45000.0, 3), (57500.0, 4), (75000.0, 5)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [0]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [73]:
# Reading CSV to Dataframe
orders = pd.read_csv('ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('OrderBreakdown.csv', encoding = 'utf-8')

In [74]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [75]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [76]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [77]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

In [80]:
orders.columns

Index(['order_id', 'order_date', 'customer_name', 'country'], dtype='object')

In [82]:
orders_break_down.columns

Index(['order_id', 'product_name', 'discount', 'sales', 'quantity',
       'category'],
      dtype='object')

#### 3. Check DataTypes 

In [78]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [79]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [91]:
# Establishing Local DB connection
db_connection = sqlite3.connect('eshop.db.sqlite')

d=db_connection.cursor()

orders.to_sql('orders_sql', con = db_connection, if_exists = 'replace', index= False)
orders_break_down.to_sql('orders_break_down_sql', con = db_connection, if_exists = 'replace', index= False)

db_connection.commit()
# Save these two dataframes as a table in sqlite

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [99]:
# Select first 10 rows of all columns in orders table

for i in d.execute('select * from orders_sql limit 10'):
    print(i)

query = 'select * from orders_sql limit 10'
       

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

('IT-2011-3647632', '2011-01-01', 'Eugene Moren', 'Sweden')
('ES-2011-4869686', '2011-01-03', 'Dorothy Dickinson', 'United Kingdom')
('ES-2011-4939443', '2011-01-04', 'Arthur Prichep', 'France')
('IT-2011-2942451', '2011-01-04', 'Grant Thornton', 'United Kingdom')
('ES-2011-3848439', '2011-01-05', 'Michael Granlund', 'France')
('ES-2011-5433855', '2011-01-07', 'Dave Poirier', 'France')
('IT-2011-4546695', '2011-01-08', 'Darren Powers', 'France')
('ES-2011-1138719', '2011-01-11', 'Eric Murdock', 'Italy')
('ES-2011-1466305', '2011-01-11', 'Mick Brown', 'Austria')
('ES-2011-4359424', '2011-01-11', 'Dorothy Dickinson', 'Spain')


,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [101]:
# Select Number of Orders for Each Customer

query = 'select count(*) from orders_sql group by customer_name'
query1 = 'select count(distinct customer_name) from orders_sql'

order_by_cust = sql.read_sql(query1, con=db_connection)
order_by_cust

,count(distinct customer_name)
0,792


#### 6. Select Number of Customers for Each Country

In [103]:
# Select Number oquery = 'select count(*) from orders_sql group by customer_name'
query2 = 'select count(*) as orders_cty from orders_sql group by country'

order_by_cty = sql.read_sql(query2, con=db_connection)
order_by_cty 

,orders_cty
0,135
1,68
2,29
3,34
4,991
5,806
6,50
7,493
8,194
9,37


#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [104]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table

query2 = 'select count(*) as orders_cty from orders_sql group by country'

order_by_cty = sql.read_sql(query2, con=db_connection)
order_by_cty 

,orders_cty
0,135
1,68
2,29
3,34
4,991
5,806
6,50
7,493
8,194
9,37


#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [108]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table


query4 = 'select discount, sales, quantity, (sales-discount) as total_price from orders_break_down_sql limit 10 '

order_by_cty = sql.read_sql(query4, con=db_connection)
order_by_cty 


,discount,sales,quantity,total_price
0,0.5,45.0,3,44.5
1,0.0,854.0,7,854.0
2,0.0,140.0,3,140.0
3,0.5,27.0,2,26.5
4,0.5,17.0,2,16.5
5,0.0,90.0,3,90.0
6,0.0,207.0,4,207.0
7,0.1,155.0,1,154.9
8,0.0,33.0,3,33.0
9,0.1,716.0,4,715.9


#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [114]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100

query3 = 'select * from orders_break_down_sql limit 10 '
query5 = 'select order_id, sales from orders_break_down_sql where sales > 100 limit 10 '

order_by_cty = sql.read_sql(query5, con=db_connection)
order_by_cty

,order_id,sales
0,ES-2011-4869686,854.0
1,ES-2011-4939443,140.0
2,ES-2011-3848439,207.0
3,ES-2011-5433855,155.0
4,ES-2011-5433855,716.0
5,IT-2011-4546695,987.0
6,IT-2011-4546695,116.0
7,ES-2011-5158390,1384.0
8,ES-2011-5158390,103.0
9,ES-2011-4359424,268.0


#### 8. Select All Customer And The Product They Have Bought

In [132]:
# Select All Customer And The Product They Have Bought

query6 = 'select customer_name, product_name \
from orders_sql join orders_break_down_sql \
on orders_sql.order_id = orders_break_down_sql.order_id limit 10'

order_by_cty = sql.read_sql(query6, con=db_connection)
order_by_cty

,customer_name,product_name
0,Eugene Moren,"Enermax Note Cards, Premium"
1,Dorothy Dickinson,"Dania Corner Shelving, Traditional"
2,Arthur Prichep,"Binney & Smith Sketch Pad, Easy-Erase"
3,Grant Thornton,"Boston Markers, Easy-Erase"
4,Grant Thornton,"Eldon Folders, Single Width"
5,Michael Granlund,"Binney & Smith Pencil Sharpener, Water Color"
6,Michael Granlund,"Sanford Canvas, Fluorescent"
7,Dave Poirier,"Accos Thumb Tacks, Assorted Sizes"
8,Dave Poirier,"Bush Floating Shelf Set, Pine"
9,Dave Poirier,"Smead Lockers, Industrial"


#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [137]:
# Select Number of 'Furniture' Orders For Each Country

query7 = 'select country, count(category) as no_of_furn \
from orders_sql join orders_break_down_sql \
on orders_sql.order_id = orders_break_down_sql.order_id \
where category = "Furniture" \
group by country'

order_by_cty = sql.read_sql(query7, con=db_connection)
order_by_cty

,country,no_of_furn
0,Austria,40
1,Belgium,20
2,Denmark,9
3,Finland,16
4,France,299
5,Germany,264
6,Ireland,12
7,Italy,136
8,Netherlands,76
9,Norway,6


#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [0]:
# Select Number of 'Furniture' Orders For The Country Denmark

query7 = 'select country, count(category) as no_of_furn \
from orders_sql join orders_break_down_sql \
on orders_sql.order_id = orders_break_down_sql.order_id \
where category = "Furniture" \
group by country'

order_by_cty = sql.read_sql(query7, con=db_connection)
order_by_cty

#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [142]:
# Select Total Sales With Discount and Without Discount for Each Country

query8 = 'select country, sum(sales) as total_sales, sum(sales-discount) as total_nett_sales \
from orders_sql join orders_break_down_sql \
on orders_sql.order_id = orders_break_down_sql.order_id \
group by country'


query9 ='select * from orders_break_down_sql'

order_by_cty = sql.read_sql(query8, con=db_connection)
order_by_cty 

,country,total_sales,total_nett_sales
0,Austria,79382.0,79382.00
1,Belgium,42320.0,42320.00
2,Denmark,7763.0,7732.50
3,Finland,20702.0,20702.00
4,France,609683.0,609536.00
5,Germany,488681.0,488591.15
6,Ireland,15998.0,15947.70
7,Italy,252742.0,252619.40
8,Netherlands,70313.0,70124.10
9,Norway,20529.0,20529.00


#### 11.A Select Total Quantity, Total Sales for Each Country

In [148]:
# Select Total Quantity, Total Sales for Each Country

query10 = 'select country, sum(sales) as total_sales, sum(quantity) as quantity \
from orders_sql join orders_break_down_sql \
on orders_sql.order_id = orders_break_down_sql.order_id \
group by country'

order_by_cty = sql.read_sql(query10, con=db_connection)
order_by_cty 

,country,total_sales,quantity
0,Austria,79382.0,973
1,Belgium,42320.0,532
2,Denmark,7763.0,204
3,Finland,20702.0,201
4,France,609683.0,7329
5,Germany,488681.0,6179
6,Ireland,15998.0,392
7,Italy,252742.0,3612
8,Netherlands,70313.0,1526
9,Norway,20529.0,261


#### 11.B Select Top 3 Country Based on Sales

In [146]:
# Select Top 3 Country Based on Sales

query10 = 'select country, sum(sales) as total_sales \
from orders_sql join orders_break_down_sql \
on orders_sql.order_id = orders_break_down_sql.order_id \
group by country \
order by total_sales desc limit 3'

order_by_cty = sql.read_sql(query10, con=db_connection)
order_by_cty 

,country,total_sales
0,France,609683.0
1,Germany,488681.0
2,United Kingdom,420497.0


#### 11.C Select Bottom 3 Country Based On Quantities

In [150]:
# Select Bottom 3 Country Based On Quantities

query11 = 'select country, sum(quantity) as quantity \
from orders_sql join orders_break_down_sql \
on orders_sql.order_id = orders_break_down_sql.order_id \
group by country \
order by quantity limit 3'

order_by_cty = sql.read_sql(query11, con=db_connection)
order_by_cty 

,country,quantity
0,Finland,201
1,Denmark,204
2,Norway,261


#### 12. Select Average Sales By Categroy For The Country 'France'

In [152]:
# Select Average Sales By Categroy For The Country 'France'

query12 = 'select country, category, avg(sales) as avg_sales \
from orders_sql join orders_break_down_sql \
on orders_sql.order_id = orders_break_down_sql.order_id \
group by country, category'

order_by_cty = sql.read_sql(query12, con=db_connection)
order_by_cty 

,country,category,avg_sales
0,Austria,Furniture,527.275000
1,Austria,Office Supplies,176.299435
2,Austria,Technology,576.297872
3,Belgium,Furniture,443.500000
4,Belgium,Office Supplies,206.864583
5,Belgium,Technology,715.315789
6,Denmark,Furniture,163.444444
7,Denmark,Office Supplies,94.461538
8,Denmark,Technology,217.333333
9,Finland,Furniture,600.125000


#### 13. Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [155]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest

query13 = 'select country, category, sum(sales) as total_sales, avg(sales) as avg_sales \
from orders_sql join orders_break_down_sql \
on orders_sql.order_id = orders_break_down_sql.order_id \
group by country, category \
order by avg_sales desc limit 1'

order_by_cty = sql.read_sql(query13, con=db_connection)
order_by_cty 

,country,category,total_sales,avg_sales
0,Switzerland,Technology,11737.0,902.846154


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



